<a href="https://colab.research.google.com/github/saurabhsingh1411/Deep_Learning_Projects/blob/main/NLP/TOP%20PROJECTS/IMDB%20dataset/IMDB_DATASET_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [2]:
data=pd.read_csv('IMDB Dataset.csv')

data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
print(data.shape)

(50000, 2)


In [4]:
data['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [5]:
data.isnull().sum()

review       0
sentiment    0
dtype: int64

In [6]:
data['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

Cleaning datasets

In [7]:
#cleaning the dataset
import re
def process_tweet(review):
    return " ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", "",review.lower()).split())

In [8]:
process_tweet(data['review'][49999])

'no one expects the star trek movies to be high art but the fans do expect a movie that is as good as some of the best episodes unfortunately this movie had a muddled implausible plot that just left me cringing this is by far the worst of the nine so far movies even the chance to watch the well known characters interact in another movie cant save this movie including the goofy scenes with kirk spock and mccoy at yosemitebr br i would say this movie is not worth a rental and hardly worth watching however for the true fan who needs to see all the movies renting this movie is about the only way youll see it even the cable channels avoid this movie'

In [12]:

processed_text=[]
for item in range(data.shape[0]):
  processed_text.append(process_tweet(data['review'][item]))

In [13]:
data['processed_review']=processed_text

data.head()

,review,sentiment,processed_review
0,One of the other reviewers has mentioned that ...,positive,one of the other reviewers has mentioned that ...
1,A wonderful little production. <br /><br />The...,positive,a wonderful little production br br the filmin...
2,I thought this was a wonderful way to spend ti...,positive,i thought this was a wonderful way to spend ti...
3,Basically there's a family where a little boy ...,negative,basically theres a family where a little boy j...
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,petter matteis love in the time of money is a ...


In [14]:
data.isnull().sum()

review              0
sentiment           0
processed_review    0
dtype: int64

In [15]:
data['sentiment']=data['sentiment'].map({'positive':1,'negative':0})

In [16]:
data['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [25]:
vocab_size = 10000
embedding_dim = 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

max_feature=10000
tokenizer=Tokenizer(num_words=max_feature,oov_token=oov_tok)
tokenizer.fit_on_texts(data['processed_review'].values)
X=tokenizer.texts_to_sequences(data['processed_review'].values)
#padded = pad_sequences(sequences,maxlen=max_length, truncating=trunc_type)

X=pad_sequences(X,maxlen=max_length, truncating=trunc_type)

In [26]:

y=pd.get_dummies(data['sentiment']).values

print(X.shape)
print(y.shape)

(50000, 120)
(50000, 2)


In [27]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(40000, 120) (40000, 2)
(10000, 120) (10000, 2)


In [39]:
import tensorflow as tf


model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 128)               245888    
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
Total params: 406,146
Trainable params: 406,146
Non-trainable params: 0
_________________________________________________________________


In [40]:
# # batch_size = 32
model.fit(X_train, y_train, epochs = 5,validation_data=(X_test ,y_test))

Epoch 1/5
1250/1250 [==============================] - 8s 6ms/step - loss: 0.5180 - accuracy: 0.7127 - val_loss: 0.3503 - val_accuracy: 0.8421
Epoch 2/5
1250/1250 [==============================] - 7s 5ms/step - loss: 0.2108 - accuracy: 0.9173 - val_loss: 0.4155 - val_accuracy: 0.8269
Epoch 3/5
1250/1250 [==============================] - 7s 5ms/step - loss: 0.0562 - accuracy: 0.9827 - val_loss: 0.6385 - val_accuracy: 0.8182
Epoch 4/5
1250/1250 [==============================] - 7s 6ms/step - loss: 0.0154 - accuracy: 0.9958 - val_loss: 1.0030 - val_accuracy: 0.8099
Epoch 5/5
1250/1250 [==============================] - 7s 6ms/step - loss: 0.0114 - accuracy: 0.9966 - val_loss: 1.0769 - val_accuracy: 0.8101
